# Analysis - Movie Subset of Netflix Data

Enhanced by Rotten Tomatoes Dataset

#### Import Libraries

In [29]:
import pandas as pd
import plotly.express as px

Create DataFrame from Netflix CSV, filtered to just movies

Create DataFrame from Rotten Tomatoes Movies CSV

In [34]:
netflix_movies_df = pd.read_csv('./data/netflix-titles.csv')
netflix_movies_df = netflix_movies_df[netflix_movies_df["type"] == 'Movie']
rotten_movies_df = pd.read_csv('./data/rotten_tomatoes_movies.csv')

Extract release_year from Rotten Tomatoes original_release_date

In [35]:
rotten_movies_df["release_year"] = rotten_movies_df["original_release_date"].str.split('-').str[0]

Add Unique identifier column to both DataFrames for merging

In [39]:
def combine_title_and_year_columns(dataframe, title_column, year_column, output_column):
  dataframe[output_column] = dataframe[title_column].str.lower() + " " + netflix_movies_df[year_column].astype(str)

# netflix_movies_df["title_and_release_year"] = netflix_movies_df["title"].str.lower() + " " + netflix_movies_df["release_year"].astype(str)
combine_columns(netflix_movies_df, "title", "release_year", output_column="title_and_release_year")
# rotten_movies_df["title_and_release_year"] = rotten_movies_df["movie_title"].str.lower() + " " + rotten_movies_df["release_year"].astype(str)
combine_columns(rotten_movies_df, "movie_title", "release_year", output_column="title_and_release_year")

Merge the above datasets, returning only those that exist in both sets

In [5]:
netflix_plus_rt_df = pd.merge(netflix_movies_df, rotten_movies_df, how="inner", on="title_and_release_year")
netflix_plus_rt_df.describe()

,release_year_x,runtime,tomatometer_rating,tomatometer_count,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
count,1252.000000,1232.000000,1251.000000,1251.000000,1248.000000,1.248000e+03,1252.000000,1252.000000,1252.000000
mean,2008.528754,107.604708,59.055156,102.459632,61.138622,4.664194e+05,24.381789,64.581470,37.818690
std,11.077374,20.597857,28.775451,85.067835,20.222044,3.310599e+06,17.706812,71.982482,39.504686
min,1954.000000,40.000000,0.000000,5.000000,8.000000,6.000000e+00,0.000000,0.000000,0.000000
25%,2004.000000,93.000000,35.000000,30.500000,46.000000,1.461500e+03,8.000000,13.000000,7.000000
50%,2012.000000,103.000000,63.000000,80.000000,63.000000,2.680450e+04,21.000000,39.000000,22.000000
75%,2017.000000,118.000000,85.500000,158.000000,77.250000,1.685648e+05,39.000000,89.000000,59.000000
max,2020.000000,213.000000,100.000000,512.000000,99.000000,3.467977e+07,65.000000,495.000000,196.000000


# Histogram - Netflix Catalogue of Movies by Release Year

Stacked by Rotten Tomatoes Tomatometer status
- ranges from Rotten to Certified Fresh
- [Rotten Tomatoes Explained](rottentomatoes.com/about)

In [28]:
netflix_plus_rt_df = netflix_plus_rt_df.dropna(subset="tomatometer_status") # Remove those without a tomatometer status
netflix_plus_rt_df = netflix_plus_rt_df[netflix_plus_rt_df["release_year_x"] >= 2000] # Remove Movies released before 2000

netflix_plus_rt_df = netflix_plus_rt_df.sort_values(by=["tomatometer_status", "release_year_x"], ascending=False)

fig = px.histogram(
  netflix_plus_rt_df,
  "release_year_x",
  title="Movies Per Year with Rotten Tomatoes Rating [2000-2020]",
  labels={"release_year_x": "Release Year"},
  color="tomatometer_status",
  color_discrete_map={
                "Rotten": "hsl(6, 78%, 66%)",
                "Fresh": "hsl(48, 89%, 60%)",
                "Certified-Fresh": "hsl(145, 61%, 59%)",
                },
)

fig.update_layout(
  barmode="group",
  xaxis_tickangle=-45,
  width=1000,
  )

In [40]:
# netflix_plus_rt_df = netflix_plus_rt_df.dropna(subset="tomatometer_status") # Remove those without a tomatometer status
# netflix_plus_rt_df = netflix_plus_rt_df[netflix_plus_rt_df["release_year_x"] >= 2000] # Remove Movies released before 2000
# is_any_fresh_filt = (netflix_plus_rt_df["tomatometer_status"] == "Certified-Fresh") | (netflix_plus_rt_df["tomatometer_status"] == "Fresh")

# netflix_plus_rt_df["Is Fresh"] = netflix_plus_rt_df[is_any_fresh_filt]

# netflix_plus_rt_df = netflix_plus_rt_df.sort_values(by=["tomatometer_status", "release_year_x"], ascending=False)


# fig = px.histogram(
#   netflix_plus_rt_df,
#   "release_year_x",
#   title="Movies Per Year with Rotten Tomatoes Rating [2000-2020]",
#   labels={"release_year_x": "Release Year"},
#   color="Is Fresh",
#   color_discrete_map={
#                 False: "hsl(6, 78%, 66%)",
#                 True: "hsl(145, 61%, 59%)",
#                 },
# )

# fig.update_layout(
#   barmode="group",
#   xaxis_tickangle=-45
#   )


In [27]:
netflix_plus_rt_df = netflix_plus_rt_df.dropna(subset="tomatometer_status")
# px.histogram(netflix_plus_rt_df, "release_year_x", color="tomatometer_status", labels={"release_year_x": "Release Year"}, color_discrete_sequence=['red', 'blue', 'green'], title="Movies Per Year with Rotten Tomatoes Rating")
netflix_plus_rt_df = netflix_plus_rt_df.sort_values(by=["tomatometer_status", "release_year_x"], ascending=False)

certified_fresh_netflix_plus_rt_df = netflix_plus_rt_df[netflix_plus_rt_df["tomatometer_status"] == "Certified-Fresh"]
fresh_netflix_plus_rt_df = netflix_plus_rt_df[netflix_plus_rt_df["tomatometer_status"] == "Fresh"]
rotten_netflix_plus_rt_df = netflix_plus_rt_df[netflix_plus_rt_df["tomatometer_status"] == "Rotten"]

px.histogram(
  netflix_plus_rt_df,
  "release_year_x",
  barmode="group",
  title="Movies Per Year rated Rotten",
  width=1000,
  labels={"release_year_x": "Release Year"},
)

In [26]:
netflix_plus_rt_df = netflix_plus_rt_df.dropna(subset="tomatometer_status")
df_grouped = netflix_plus_rt_df.groupby("release_year_x").count().reset_index()
px.bar(df_grouped,
      x='release_year_x',
      y='title',
      title='Test',
      color='tomatometer_status',
      barmode="stack",
      color_discrete_map={
        "Rotten": "red",
        "Fresh": "yellow",
        "Certified-Fresh": "green",
        },
      width=1000,
      labels={
        "release_year_x": "Release Year",
        "title": "Count",
        }
      )

  ### Plan for Wednesday 15/06/2022
  - Refactor into functions [Timebox: 1hr => 11:30]
  - Python-Slugify & Peter Norvig spelling => pre-merge
  - **5** different types of graph

In [41]:
Test change

SyntaxError: invalid syntax (1702171434.py, line 1)